# Scoping du projet

Ce projet vise à révolutionner la manière dont nous interagissons avec la parole en développant un modèle de Reconnaissance Automatique de la Parole (ASR) avancé et en y ajoutant une couche d'Analyse de Sentiment pour extraire des informations émotionnelles précieuses à partir des transcriptions générées. Cette approche marie la technologie de traitement automatique de la parole à l'intelligence émotionnelle, offrant un large éventail d'applications potentielles.

En somme, le projet consiste à développer un modèle de Reconnaissance Automatique de la Parole (ASR), puis effectuer une Analyse de Sentiment sur la transcription générée par le modèle ASR.

La première partie du projet se concentrera sur la mise en place du modèle ASR. Nous avons utilisé un modèle français sur le Hub de Huggingface en inférence pour transcrire des enregistrements audio en texte.

Dans la deuxième partie du projet, nous avons d'abord fine-tuné un modèle de classification BERT (positif ou négatif ) entrainé sur le jeu de données : https://www.kaggle.com/datasets/djilax/allocine-french-movie-reviews.
Ensuite, nous avons effectué une analyse de sentiment sur la transcription générée du modèle ASR.

1. **Objectif du projet** :
   - L'objectif principal est de construire un modèle de Reconnaissance Automatique de la Parole (ASR), puis effectuer une Analyse de Sentiment sur la transcription générée par le modèle ASR.

2. **Tâche** :
   - modèle d'ASR : utiliser un modèle français sur le Hub de Huggingface en inférence pour transcrire des enregistrements audio en texte.
   - modèle de classification : fine-tuner un modèle de classification BERT (positif ou négatif ) entrainé sur le jeu de données : https://www.kaggle.com/datasets/djilax/allocine-french-movie-reviews.

3. **Métriques d'évaluation** :
   - Les métriques d'évaluation appropriées pour cette tâche pourraient inclure la précision, le rappel, la F1-score, et la matrice de confusion.

4. **Collecte et préparation des données** :
   - Prétraitement : Nettoyer le texte en retirant les caractères spéciaux, en normalisant la casse, etc.
   - Diviser le jeu de données en ensembles d'entraînement, de validation et de test.

5. **Architecture du modèle** :
   - pour l'ASR, nous allons utiliser un modèle français en inférence sur le Hub de Huggingface.
   - Pour l'analyse de sentiment, nous allons fine-tuner un modèle de classification BERT entrainé sur notre jeu de données.

6. **Entraînement et évaluation du modèle** :
   - Entraînez le modèle sur l'ensemble d'entraînement et ajustez les hyperparamètres pour optimiser les performances.
   - Évaluez le modèle sur l'ensemble de validation pour surveiller le surapprentissage et ajuster les paramètres en conséquence.

7. **Tests finaux** :
   - Évaluez le modèle final sur l'ensemble de test pour obtenir une évaluation impartiale de ses performances.

8. **Communication des résultats** :
   - Présentez les performances du modèle à l'aide de métriques appropriées dans un rapport ou une présentation.

9. **Itérations** :
   - En fonction des performances du modèle, effectuez des itérations pour ajuster les hyperparamètres, essayer différentes architectures et améliorer les résultats.

# Data

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Charger le jeu de données
df = pd.read_csv('/content/drive/Othercomputers/Mon ordinateur/DIT MASTER1 2022/MASTER 2/DEEP LEARNING/examen/dataset/train.csv')

In [ ]:
df.head()

,Unnamed: 0,film-url,review,polarity
0,0,http://www.allocine.fr/film/fichefilm-135259/c...,Si vous cherchez du cinéma abrutissant à tous ...,0
1,1,http://www.allocine.fr/film/fichefilm-172430/c...,"Trash, re-trash et re-re-trash...! Une horreur...",0
2,2,http://www.allocine.fr/film/fichefilm-15105/cr...,"Et si, dans les 5 premières minutes du film, l...",0
3,3,http://www.allocine.fr/film/fichefilm-188629/c...,Mon dieu ! Quelle métaphore filée ! Je suis ab...,0
4,4,http://www.allocine.fr/film/fichefilm-23514/cr...,"Premier film de la saga Kozure Okami, ""Le Sabr...",1


In [ ]:
len(df)

160000

In [ ]:
# Compter le nombre de labels distincts
num_unique_labels = df['polarity'].nunique()
print(num_unique_labels)

2


In [ ]:
# Afficher les labels distincts
unique_labels = df['polarity'].unique()
print(f"Labels distincts : {unique_labels}")

Labels distincts : [0 1]


# Modeling

In [ ]:
#!pip install transformers
#!pip install sentencepiece
#!pip install wandb

In [ ]:
from transformers import AutoTokenizer, BertModel, AdamW, PreTrainedModel, BertForPreTraining, AutoModelForSequenceClassification
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import pandas as pd
import torch.nn.functional as F
from huggingface_hub import PyTorchModelHubMixin


from tqdm import tqdm

from sklearn.model_selection import train_test_split

import wandb

In [ ]:
config = {
    "num_epochs" : 5,
    "max_length" : 80,
    "num_classes" : 2,
    "learning_rate" : 2e-5,
    "batch_size" : 100,
    "shuffle" : True,
    "model_name" : 'nlptown/bert-base-multilingual-uncased-sentiment',
    "device" : torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    "train_file" : "/content/drive/Othercomputers/Mon ordinateur/DIT MASTER1 2022/MASTER 2/DEEP LEARNING/examen/dataset/train.csv",
    "valid_file" : "/content/drive/Othercomputers/Mon ordinateur/DIT MASTER1 2022/MASTER 2/DEEP LEARNING/examen/dataset/valid.csv"
}

In [ ]:
class MyDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length):
        self.df = pd.read_csv(csv_file)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        item = self.df.iloc[index]
        sentence = item['review']
        label = item['polarity']

        inputs = self.tokenizer(sentence, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')

        return {
            "input_ids" : inputs['input_ids'],
            "attention_mask" : inputs['attention_mask'],
            "label" : torch.tensor(label)
        }

In [ ]:
# dataloader
def dataloader(dataset, batch_size, shuffle):
    return DataLoader(dataset=dataset, batch_size= batch_size, shuffle=shuffle)

In [ ]:
class CustomModel(nn.Module, PyTorchModelHubMixin):
    def __init__(self, model_name, num_classes):
        super(CustomModel, self).__init__()
        self.pretrained_model = BertModel.from_pretrained(model_name) # 768 corresponds to the BERT's output size
        self.classifier = nn.Linear(768, num_classes)   # MLP

    def forward(self, input_ids, attention_mask):
        output = self.pretrained_model(input_ids=input_ids, attention_mask=attention_mask) # (bacth 768)
        output = self.classifier(output.last_hidden_state)
        return output

In [ ]:
# training step
def train_step(model, train_loader, optimizer, loss_fn, device, num_classes):
    model.train()

    total_loss = 0

    for data in tqdm(train_loader, total = len(train_loader)):
        inputs = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        targets = data['label'].to(device)

        optimizer.zero_grad()
        output = model(inputs.squeeze(1), attention_mask)
        loss = loss_fn(output, F.one_hot(targets, num_classes))

        # backward
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return  total_loss / len(train_loader)

In [ ]:
def validation_step(model, validation_loader, loss_fn, device, num_classes):
    total_loss = 0
    correct_prediction = 0

    with torch.no_grad():
        for data in tqdm(validation_loader, total = len(validation_loader)):
            input_ids = data['input_ids'].squeeze(1).to(device)
            attention_mask = data['attention_mask'].to(device)
            targets = data['label'].to(device)

            output = model(input_ids = input_ids, attention_mask = attention_mask)
            loss = loss_fn(output, F.one_hot(targets, num_classes))

            total_loss += loss.item()

            pred = torch.max(torch.softmax(output, dim=1), dim = 1)
            correct_prediction += torch.sum(pred.indices==F.one_hot(targets, num_classes))


        return  total_loss / len(validation_loader), 100*correct_prediction/len(validation_loader)

In [ ]:
# main
def main():
    wandb.init('bert-base-sentiment-classification')

    tokenizer = AutoTokenizer.from_pretrained(config['model_name'])
    # dataset
    train_dataset = MyDataset(csv_file=config['train_file'], tokenizer=tokenizer, max_length=config['max_length'])
    validation_dataset = MyDataset(csv_file=config['valid_file'], tokenizer=tokenizer, max_length=config['max_length'])

    #train_dataset, validation_dataset = train_test_split(dataset, test_size=0.2)

    # trainloader
    train_loader = dataloader(train_dataset, batch_size=config['batch_size'], shuffle=True)
    validation_loader = dataloader(validation_dataset, batch_size=config['batch_size'], shuffle=False)

    # model
    model = CustomModel(model_name=config['model_name'], num_classes=config['num_classes'])
    model.to(config['device'])

    loss_fn = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=config['learning_rate'])

    for epoch in range(config['num_epochs']):
        loss_train = train_step(model, train_loader, optimizer, loss_fn, config['device'], config['num_classes'])
        loss_validatiton, accuracy = validation_step(model, validation_loader, loss_fn, config['device'], config['num_classes'])

        wandb.log({
            "loss_train":loss_train,
            "loss_validatiton":loss_validatiton,
            "accuracy":accuracy
            })
    # push model to hub
    model.push_to_hub("billfass/bert-base-sentiment-classification")
    tokenizer.push_to_hub("billfass/bert-base-sentiment-classification")
    model.pretrained_model.config.push_to_hub("billfass/bert-base-sentiment-classification")

In [ ]:
!git config --global credential.helper store
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Your token has been saved in your

In [ ]:
main()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 200/200 [00:46<00:00,  4.32it/s]


pytorch_model.bin:   0%|          | 0.00/670M [00:00<?, ?B/s]

#Deployment